<a href="https://colab.research.google.com/github/YoshiyukiKono/langchain_for_beginners/blob/main/01_RAG_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PDFファイルを使ったRAG

https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf

## 準備

### Astra DBの準備
 1. Astraで、新しいベクトル検索対応データベース(`demo`)を作成します（データベース名はConnect Bundleファイルに関係します）。
 2. キースペース (`langchain`)を作成します（キースペース名はデータベース作成時のダイアログにて指定します）
 3. `Database Administrator`権限を持つトークンを生成します。

 上記で作成済みのデータベース/キースペースに、このチュートリアルの中で、テーブルとインデックスを作成します。


### パッケージインストール

In [1]:
!pip install openai==1.8.0 tiktoken==0.5.2 cohere==4.44
!pip install langchain==0.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.3/222.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  Attempting uninstall: importlib_metadata
    Found existing installation: importlib-metadata 7.0.1
    Uninstalling importlib-metadata-7.0.1:
      Successfully uninstalled importlib-metadata-7.0.1
ERROR: pip's dependency resolver does not cur

In [2]:
!pip install --upgrade astrapy==0.7.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.26.0
    Uninstalling httpx-0.26.0:
      Successfully uninstalled httpx-0.26.0


In [3]:
!pip install pypdf==3.17.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.4/277.4 kB 2.7 MB/s eta 0:00:00


## OpenAI 接続

In [4]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


入力されたキーによる接続確認

In [5]:
import openai
from openai import OpenAI

client = OpenAI()
client.models.list()

SyncPage[Model](data=[Model(id='curie-search-query', created=1651172509, object='model', owned_by='openai-dev'), Model(id='babbage-search-query', created=1651172509, object='model', owned_by='openai-dev'), Model(id='dall-e-3', created=1698785189, object='model', owned_by='system'), Model(id='babbage-search-document', created=1651172510, object='model', owned_by='openai-dev'), Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-0301', created=1677649963, object='model', owned_by='openai'), Model(id='text-embedding-ada-002', created=1671217299, object='model', owned_by='openai-internal'), Model(id='davinci-search-query', created=1651172505, object='model', owned_by='openai-dev'), Model(id='tts-1-hd-1106', created=1699053533, object='model', owned_by='system'), Model(id='gpt-4-0613', created=1686588896, object='model', owned_by='openai'), Model(id='tts-1-hd', created=1699046015, object='model', owned_by='system'), Model(id='gpt-4', created=

【下記セルを実行すると入力欄が表示されます】
AstraCS:から始まるトークンを設定します。

In [6]:
import getpass

YOUR_TOKEN = getpass.getpass()

··········


【下記セルを実行すると入力欄が表示されます】
あなたのAstra環境に表示されているサンプルプログラムからAPIエンドポイントの値をコピーして、設定します。

In [7]:
API_ENDPOINT = input()

https://95154b76-8356-4fb2-89f3-57779b833dfb-us-east1.apps.astra.datastax.com


接続確認

In [8]:
from astrapy.db import AstraDB

# Initialization
db = AstraDB(
  token=YOUR_TOKEN,
  api_endpoint=API_ENDPOINT)

print(f"Connected to Astra DB: {db.get_collections()}")

Connected to Astra DB: {'status': {'collections': ['book_collection', 'vector_test']}}


## LangChainの利用



In [9]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Cassandra
from langchain.document_loaders import TextLoader

### PDFファイルの準備

このリポジトリの`data`フォルダにあるファイル(`DataStax_Vector-Search.pdf`)を、Colab環境にアップロードしてください。

このアップロードしたファイルを下記で読み込みます。

In [10]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("./DataStax_Vector-Search.pdf")
pages = loader.load_and_split()

In [11]:
print(len(pages))
print(pages[0])
print(pages[len(pages)-1])

24
page_content='ホワイトペーパー\n生成 AIアプリ\nのための\nベクトル検索\nAIアプリケーション開発にベクトル検索を活用す\nるための開発者/アーキテクト向けガイド\nこのドキュメントは、生成 AIアプリケーションを企業独\n自のデータと組み合わせて、設計・構築しようとしてい\nる全ての方のためのガイドです。組織が理解すべき重\n要な概念と考慮事項を取り扱うだけでなく、ベクトル検\n索を用いて、 LLMの持つ機能を大幅に拡張するため\nのシンプルで強力なアプローチについても解説しま\nす。' metadata={'source': './DataStax_Vector-Search.pdf', 'page': 0}
page_content='20. Karpathy, Andrej. “Andrej Karpathy on Twitter: "The hottest new programming language is\nEnglish."” Twitter , 2023年 1月24日 ,https://twitter .com/karpathy/status/1617979122625712128\n21. “MTEB Leaderboard - a Hugging Face Space by mteb.” Hugging Face,\nhttps://huggingface.co/spaces/mteb/leaderboard\n22.https://platform.openai.com/docs/guides/embeddings/types-of-embedding-models\n23. Instructor Text Embedding, https://instructor-embedding.github.io/\n24. “Libraries - OpenAI API.” Platform OpenAI, https://platform.openai.com/docs/libraries\n25. “Vertex AI.” Google Cloud, https://cloud.google.com/vertex-ai\n©2023 DataStax Inc.、全著作権所有。 DataStax は、米国およびその他

In [12]:
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=10)
docs = text_splitter.split_documents(pages)

In [13]:
print(len(docs))
print(docs[0])
print(docs[len(docs)-1])

24
page_content='ホワイトペーパー\n生成 AIアプリ\nのための\nベクトル検索\nAIアプリケーション開発にベクトル検索を活用す\nるための開発者/アーキテクト向けガイド\nこのドキュメントは、生成 AIアプリケーションを企業独\n自のデータと組み合わせて、設計・構築しようとしてい\nる全ての方のためのガイドです。組織が理解すべき重\n要な概念と考慮事項を取り扱うだけでなく、ベクトル検\n索を用いて、 LLMの持つ機能を大幅に拡張するため\nのシンプルで強力なアプローチについても解説しま\nす。' metadata={'source': './DataStax_Vector-Search.pdf', 'page': 0}
page_content='20. Karpathy, Andrej. “Andrej Karpathy on Twitter: "The hottest new programming language is\nEnglish."” Twitter , 2023年 1月24日 ,https://twitter .com/karpathy/status/1617979122625712128\n21. “MTEB Leaderboard - a Hugging Face Space by mteb.” Hugging Face,\nhttps://huggingface.co/spaces/mteb/leaderboard\n22.https://platform.openai.com/docs/guides/embeddings/types-of-embedding-models\n23. Instructor Text Embedding, https://instructor-embedding.github.io/\n24. “Libraries - OpenAI API.” Platform OpenAI, https://platform.openai.com/docs/libraries\n25. “Vertex AI.” Google Cloud, https://cloud.google.com/vertex-ai\n©2023 DataStax Inc.、全著作権所有。 DataStax は、米国およびその他

別のクラス（`RecursiveCharacterTextSplitter`）を使って、より細かいチャンクに分割してみる。

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=0)

texts = text_splitter.split_documents(pages)

In [19]:
print(len(texts))
print(texts[0])
print(texts[len(texts)-1])

82
page_content='ホワイトペーパー\n生成 AIアプリ\nのための\nベクトル検索\nAIアプリケーション開発にベクトル検索を活用す\nるための開発者/アーキテクト向けガイド\nこのドキュメントは、生成 AIアプリケーションを企業独\n自のデータと組み合わせて、設計・構築しようとしてい\nる全ての方のためのガイドです。組織が理解すべき重\n要な概念と考慮事項を取り扱うだけでなく、ベクトル検\n索を用いて、 LLMの持つ機能を大幅に拡張するため\nのシンプルで強力なアプローチについても解説しま\nす。' metadata={'source': './DataStax_Vector-Search.pdf', 'page': 0}
page_content='23. Instructor Text Embedding, https://instructor-embedding.github.io/\n24. “Libraries - OpenAI API.” Platform OpenAI, https://platform.openai.com/docs/libraries\n25. “Vertex AI.” Google Cloud, https://cloud.google.com/vertex-ai\n©2023 DataStax Inc.、全著作権所有。 DataStax は、米国およびその他の国における DataStax, Inc. およびそ\nの子会社の登録商標です。\n24' metadata={'source': './DataStax_Vector-Search.pdf', 'page': 23}


In [ ]:
documents=texts

## ベクトルストアの構築

OpenAIのエンベディングモデルではなく、（連続）利用制限のない、オープンなエンベディングモデルを利用します。

In [20]:

!pip install sentence_transformers==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.9 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=4a43a9c9a257fc2af759fd5ab62fc775cff14b117d890bad6dc92dbde46caaa3
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [21]:

from langchain.embeddings import HuggingFaceEmbeddings

text = "テスト文字列"

embeddings = HuggingFaceEmbeddings()
query_result = embeddings.embed_query(text)
doc_result = embeddings.embed_documents([text])

print(len(query_result))
print(query_result)

print(len(doc_result[0]))
print(doc_result)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

768
[0.02339586801826954, -0.051528092473745346, -0.01799503155052662, 0.010700330138206482, 0.039436064660549164, 0.027511082589626312, -0.03647975996136665, 0.06465698033571243, 0.013887857086956501, 0.018204664811491966, 0.03117774985730648, 0.015076836571097374, 0.07378007471561432, 0.029201000928878784, -0.043200548738241196, -0.07707522064447403, -0.0067560975439846516, 0.0345374271273613, 0.045760996639728546, -0.02631659060716629, 0.015634726732969284, 0.012039588764309883, 0.015451109036803246, -0.014117742888629436, 0.015340771526098251, -0.044638700783252716, 0.012640316039323807, -0.012221626937389374, 0.029994942247867584, 0.0002732427674345672, 0.047916773706674576, -0.022249307483434677, -0.0039941188879311085, 0.008768660947680473, 1.510015181338531e-06, -0.017982913181185722, -0.021689971908926964, -0.026938535273075104, -0.018404435366392136, -0.04279183968901634, 0.041685618460178375, -0.0442582331597805, -0.006897759158164263, 0.016141779720783234, 0.027121547609567

In [24]:
from langchain_community.vectorstores import AstraDB
vstore = AstraDB(
    embedding=embeddings,
    collection_name="pdf_collection",
    token=YOUR_TOKEN,
    api_endpoint=API_ENDPOINT,
)


上記の操作により、下記のテーブルとインデックスが作成されます。
```
CREATE TABLE default_keyspace.pdf_collection (
    key frozen<tuple<tinyint, text>> PRIMARY KEY,
    array_contains set<text>,
    array_size map<text, int>,
    doc_json text,
    exist_keys set<text>,
    query_bool_values map<text, tinyint>,
    query_dbl_values map<text, decimal>,
    query_null_values set<text>,
    query_text_values map<text, text>,
    query_timestamp_values map<text, timestamp>,
    query_vector_value vector<float, 768>,
    tx_id timeuuid
) WITH additional_write_policy = '99p'
    AND bloom_filter_fp_chance = 0.01
    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
    AND comment = ''
    AND compaction = {'class': 'org.apache.cassandra.db.compaction.UnifiedCompactionStrategy'}
    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
    AND crc_check_chance = 1.0
    AND default_time_to_live = 0
    AND gc_grace_seconds = 864000
    AND max_index_interval = 2048
    AND memtable_flush_period_in_ms = 0
    AND min_index_interval = 128
    AND read_repair = 'BLOCKING'
    AND speculative_retry = '99p';
CREATE CUSTOM INDEX pdf_collection_array_contains ON default_keyspace.pdf_collection (values(array_contains)) USING 'StorageAttachedIndex';
CREATE CUSTOM INDEX pdf_collection_array_size ON default_keyspace.pdf_collection (entries(array_size)) USING 'StorageAttachedIndex';
CREATE CUSTOM INDEX pdf_collection_exists_keys ON default_keyspace.pdf_collection (values(exist_keys)) USING 'StorageAttachedIndex';
CREATE CUSTOM INDEX pdf_collection_query_bool_values ON default_keyspace.pdf_collection (entries(query_bool_values)) USING 'StorageAttachedIndex';
CREATE CUSTOM INDEX pdf_collection_query_dbl_values ON default_keyspace.pdf_collection (entries(query_dbl_values)) USING 'StorageAttachedIndex';
CREATE CUSTOM INDEX pdf_collection_query_null_values ON default_keyspace.pdf_collection (values(query_null_values)) USING 'StorageAttachedIndex';
CREATE CUSTOM INDEX pdf_collection_query_text_values ON default_keyspace.pdf_collection (entries(query_text_values)) USING 'StorageAttachedIndex';
CREATE CUSTOM INDEX pdf_collection_query_timestamp_values ON default_keyspace.pdf_collection (entries(query_timestamp_values)) USING 'StorageAttachedIndex';
CREATE CUSTOM INDEX pdf_collection_query_vector_value ON default_keyspace.pdf_collection (query_vector_value) USING 'StorageAttachedIndex' WITH OPTIONS = {'similarity_function': 'cosine'};
```

Astra DBの「CQL Console」タブで、次のコマンドを実行して上記のDDLを確認することができます。

```
desc default_keyspace.pdf_collection;
```


In [26]:
print(len(docs))

24


In [25]:
inserted_ids = vstore.add_documents(docs)


下記のように、登録されたデータの件数を確認することができます。

```
select count(key) from default_keyspace.pdf_collection;
```

```
token@cqlsh:default_keyspace> select count(key) from default_keyspace.pdf_collection;

 system.count(key)
-------------------
                24

(1 rows)
```

まず、以下では、Astra DBベクトルデータベースに対する、直接のベクトル検索実行を試してみます。

In [27]:
query = "RAGのユースケース"
docs = vstore.similarity_search(query)
print(len(docs))
for doc in docs:
  print(doc)

4
page_content='20. Karpathy, Andrej. “Andrej Karpathy on Twitter: "The hottest new programming language is\nEnglish."” Twitter , 2023年 1月24日 ,https://twitter .com/karpathy/status/1617979122625712128\n21. “MTEB Leaderboard - a Hugging Face Space by mteb.” Hugging Face,\nhttps://huggingface.co/spaces/mteb/leaderboard\n22.https://platform.openai.com/docs/guides/embeddings/types-of-embedding-models\n23. Instructor Text Embedding, https://instructor-embedding.github.io/\n24. “Libraries - OpenAI API.” Platform OpenAI, https://platform.openai.com/docs/libraries\n25. “Vertex AI.” Google Cloud, https://cloud.google.com/vertex-ai\n©2023 DataStax Inc.、全著作権所有。 DataStax は、米国およびその他の国における DataStax, Inc. およびそ\nの子会社の登録商標です。\n24' metadata={'source': './DataStax_Vector-Search.pdf', 'page': 23}
page_content='生成 AIの登場は、プロダクトを利用する方法だけでなく、プロダクトを構築する方法にも変化を与えています。\nOpenAIが 2022年 11月 30日に ChatGPTを発表して以来わずか数ヶ月の間に生成 AIへの関心が世界を席巻しま\nした。この記事を書いている時点で、マッキンゼーは、今や生成 AIの価値は世界経済全体で 2兆4,000億ドルから 4\n兆2,000億ドル規模になる可能性がある

## Agent

LangChainのエージェントを試してみます。

In [28]:

retriever = vstore.as_retriever(search_kwargs={"k":1})

In [29]:

from langchain.agents.agent_toolkits import create_retriever_tool, create_conversational_retrieval_agent
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.callbacks import StreamlitCallbackHandler
from langchain.schema import BaseRetriever, Document, SystemMessage

In [30]:
from langchain.globals import set_debug, set_verbose

set_debug(True)
set_verbose(False)

In [33]:
from langchain.llms import OpenAI
from langchain.chains import LLMMathChain
from langchain.agents import Tool
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

retriever_tool = create_retriever_tool(
        retriever, "my_retrevier", "Useful when searching for the information about technologies")
tools=[retriever_tool]

system_message_content = """
    Please answer using the given my_retreiver.
    """
system_message_content = f"{system_message_content} All the responses should be in Japanese language."
system_message = SystemMessage(content=system_message_content)

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, system_message=system_message, verbose=True, max_tokens_limit=500)

In [34]:
agent.run("DATASTAXについて説明して")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "DATASTAXについて説明して"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "DATASTAXについて説明して",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Answer the following questions as best you can. You have access to the following tools:\n\nmy_retrevier: Useful when searching for the information about technologies\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [my_retrevier]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final

'DATASTAXは、エンベディングデータの生成と保存に用いられるツールであり、データソースやアプリケーションの面からチャンク化の戦略を立てて、近似最近傍探索（ANN）を利用した高速ベクトル検索が可能になる。また、AstraDBのベクトル検索機能は、高い可用性と柔軟なスケールを有し、ベクトルとともに、そのベクトルに関連するメタデータを保存しておくことも容易である。'

上記で利用した下記のコードの`k`の数を変更することによって、結果が変わることを確認することもできます。

```
retriever = vstore.as_retriever(search_kwargs={"k":1})
```

In [44]:
retriever = vstore.as_retriever(search_kwargs={"k":3})
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, system_message=system_message, verbose=True, max_tokens_limit=400)
agent.run("DATASTAXについて説明して")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "DATASTAXについて説明して"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "DATASTAXについて説明して",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Answer the following questions as best you can. You have access to the following tools:\n\nmy_retrevier: Useful when searching for the information about technologies\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [my_retrevier]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final

'DataStaxは、高速ベクトル検索を行うためのデータベースであり、データストアにデータを保存することで、データに関連するメタデータを用いた操作も可能である。'